In [55]:
import pandas as pd
import py_entitymatching as em
import py_stringmatching as sm
import numpy as np

In [142]:
A = em.read_csv_metadata('api_names.csv', key='PLAYER_ID')
B = em.read_csv_metadata('espn_names.csv', key='player_id')

In [59]:
print str(len(A))
print str(len(B))

469
447


In [60]:
em.get_key(A), em.get_key(B)

('PLAYER_ID', 'player_id')

In [61]:
A.head(2)

,Unnamed: 0,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,USG_PCT_RANK,PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,CFID,CFPARAMS
0,0,1627773,AJ Hammons,1610612742,DAL,24.0,17,3,14,0.176,...,292,244,421,413,416,434,436,320,5,"1627773,1610612742"
1,1,201166,Aaron Brooks,1610612754,IND,32.0,48,27,21,0.563,...,182,327,347,276,258,308,287,336,5,"201166,1610612754"


In [62]:
B.head(2)

,Unnamed: 0,player_id,player_name
0,0,2991178,a.j.-hammons
1,1,3192,aaron-brooks


In [50]:
type(B)

pandas.core.frame.DataFrame

In [63]:
lev = sm.Levenshtein()

In [82]:
B.loc[0]['player_name'].lower()

'a.j.-hammons'

In [83]:
A.loc[0]['PLAYER_NAME'].lower()

'aj hammons'

In [81]:
lev.get_sim_score(A.loc[0]['PLAYER_NAME'].lower(), B.loc[0]['player_name'].lower())

0.75

In [70]:
B.loc[25]['player_name']

'arron-afflalo'

In [72]:
B.loc[B['player_name'] == 'rondae-hollis-jefferson']

,Unnamed: 0,player_id,player_name
364,364,3064291,rondae-hollis-jefferson


In [74]:
A.loc[A['PLAYER_NAME'] == 'Rondae Hollis-Jefferson']

,Unnamed: 0,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,USG_PCT_RANK,PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,CFID,CFPARAMS
384,384,1626178,Rondae Hollis-Jefferson,1610612751,BKN,22.0,55,9,46,0.164,...,223,9,194,197,187,223,210,284,5,"1626178,1610612751"


In [79]:
print '{0:25} ==> {1:25s}'.format(A.loc[384]['PLAYER_NAME'].lower(),
                                 B.loc[364]['player_name'].lower())
print '{0:25} ==> {1:25s}'.format(A.loc[0]['PLAYER_NAME'].lower(),
                                 B.loc[0]['player_name'].lower())

rondae hollis-jefferson   ==> rondae-hollis-jefferson  
aj hammons                ==> a.j.-hammons             


In [84]:
for nba_index, nba_row in A.iterrows():
    player_name_lower_nba = A.loc[nba_index]['PLAYER_NAME'].lower()
    for espn_index, espn_row in B.iterrows():
        player_name_lower_espn = B.loc[espn_index]['player_name'].lower()
        if(lev.get_sim_score(player_name_lower_nba, player_name_lower_espn) > 0.7):
            print '{0:25} ==> {1:25s}'.format(player_name_lower_nba, player_name_lower_espn)

aj hammons                ==> a.j.-hammons             
aaron brooks              ==> aaron-brooks             
aaron gordon              ==> aaron-gordon             
adreian payne             ==> adreian-payne            
al horford                ==> al-horford               
al jefferson              ==> al-jefferson             
al-farouq aminu           ==> al-farouq-aminu          
alan anderson             ==> alan-anderson            
alan anderson             ==> ryan-anderson            
alan williams             ==> alan-williams            
alec burks                ==> alec-burks               
alex abrines              ==> alex-abrines             
alex len                  ==> alex-len                 
alexis ajinca             ==> alexis-ajinca            
allen crabbe              ==> allen-crabbe             
amir johnson              ==> amir-johnson             
andre drummond            ==> andre-drummond           
andre iguodala            ==> andre-iguodala    

KeyboardInterrupt: 

In [132]:
similarity_matrix = pd.DataFrame(columns=['nba_id', 'score', 'nba_name'])

similarity_matrix = similarity_matrix.append(B, ignore_index=True)
similarity_matrix = similarity_matrix.fillna(0.0)
similarity_matrix['nba_id'] = similarity_matrix['nba_id'].astype('int')
similarity_matrix['nba_name'] = similarity_matrix['nba_name'].astype('str')
similarity_matrix['player_id'] = similarity_matrix['player_id'].astype('int')
similarity_matrix = similarity_matrix.drop('Unnamed: 0', 1)
similarity_matrix.head()

,nba_id,nba_name,player_id,player_name,score
0,0,0.0,2991178,a.j.-hammons,0.0
1,0,0.0,3192,aaron-brooks,0.0
2,0,0.0,3064290,aaron-gordon,0.0
3,0,0.0,2531100,adreian-payne,0.0
4,0,0.0,6569,alan-anderson,0.0


In [136]:
for nba_index, nba_row in A.iterrows():
    player_name_lower_nba = A.loc[nba_index]['PLAYER_NAME'].lower()
    for espn_index, espn_row in B.iterrows():
        player_name_lower_espn = B.loc[espn_index]['player_name'].lower()
        score = lev.get_sim_score(player_name_lower_nba, player_name_lower_espn)
        if (score > 0.7):
#             print 'score = ' + str(score)
#             print 'current score = ' + str(similarity_matrix.loc[espn_index]['score'])
            if (score > similarity_matrix.loc[espn_index]['score']):
                similarity_matrix.loc[espn_index]['player_name']
#                 print score
#                 print similarity_matrix.loc[espn_index]['player_name']
                similarity_matrix.set_value(espn_index, 'nba_id', nba_row['PLAYER_ID'])
                similarity_matrix.set_value(espn_index, 'score', score)
                similarity_matrix.set_value(espn_index, 'nba_name', nba_row['PLAYER_NAME'])
                # dfmi.loc[:,('one','second')] = value
    print nba_index

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [137]:
similarity_matrix

,nba_id,nba_name,player_id,player_name,score
0,1627773,AJ Hammons,2991178,a.j.-hammons,0.750000
1,201166,Aaron Brooks,3192,aaron-brooks,0.916667
2,203932,Aaron Gordon,3064290,aaron-gordon,0.916667
3,203940,Adreian Payne,2531100,adreian-payne,0.923077
4,101187,Alan Anderson,6569,alan-anderson,0.923077
5,1626210,Alan Williams,2579326,alan-williams,0.923077
6,202692,Alec Burks,6429,alec-burks,0.900000
7,203518,Alex Abrines,2995702,alex-abrines,0.916667
8,201582,Alexis Ajinca,3410,alexis-ajinca,0.923077
9,203458,Alex Len,2596107,alex-len,0.875000


In [144]:
for index, row in similarity_matrix.iterrows():
    A.loc[A.PLAYER_ID == row['nba_id'], 'PICTURE_ID'] = row['player_id']

In [145]:
A.head()

,Unnamed: 0,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,CFID,CFPARAMS,PICTURE_ID
0,0,1627773,AJ Hammons,1610612742,DAL,24.0,17,3,14,0.176,...,244,421,413,416,434,436,320,5,"1627773,1610612742",2991178.0
1,1,201166,Aaron Brooks,1610612754,IND,32.0,48,27,21,0.563,...,327,347,276,258,308,287,336,5,"201166,1610612754",3192.0
2,2,203932,Aaron Gordon,1610612753,ORL,21.0,59,22,37,0.373,...,231,268,91,77,119,105,273,5,"203932,1610612753",3064290.0
3,3,1626151,Aaron Harrison,1610612766,CHA,22.0,5,2,3,0.400,...,456,457,462,451,462,456,462,5,"1626151,1610612766",NaN
4,4,203940,Adreian Payne,1610612750,MIN,26.0,12,4,8,0.333,...,425,298,398,401,346,356,257,5,"203940,1610612750",2531100.0


In [146]:
A

,Unnamed: 0,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,CFID,CFPARAMS,PICTURE_ID
0,0,1627773,AJ Hammons,1610612742,DAL,24.0,17,3,14,0.176,...,244,421,413,416,434,436,320,5,"1627773,1610612742",2991178.0
1,1,201166,Aaron Brooks,1610612754,IND,32.0,48,27,21,0.563,...,327,347,276,258,308,287,336,5,"201166,1610612754",3192.0
2,2,203932,Aaron Gordon,1610612753,ORL,21.0,59,22,37,0.373,...,231,268,91,77,119,105,273,5,"203932,1610612753",3064290.0
3,3,1626151,Aaron Harrison,1610612766,CHA,22.0,5,2,3,0.400,...,456,457,462,451,462,456,462,5,"1626151,1610612766",NaN
4,4,203940,Adreian Payne,1610612750,MIN,26.0,12,4,8,0.333,...,425,298,398,401,346,356,257,5,"203940,1610612750",2531100.0
5,5,201143,Al Horford,1610612738,BOS,30.0,49,32,17,0.653,...,248,95,81,76,68,70,202,5,"201143,1610612738",3213.0
6,6,2744,Al Jefferson,1610612754,IND,32.0,57,29,28,0.509,...,332,46,122,151,144,173,88,5,"2744,1610612754",2389.0
7,7,202329,Al-Farouq Aminu,1610612757,POR,26.0,39,16,23,0.410,...,193,220,250,214,207,159,415,5,"202329,1610612757",4248.0
8,8,101187,Alan Anderson,1610612746,LAC,34.0,25,16,9,0.640,...,396,441,378,365,389,377,424,5,"101187,1610612746",6569.0
9,9,1626210,Alan Williams,1610612756,PHX,24.0,26,5,21,0.192,...,213,79,336,356,310,353,35,5,"1626210,1610612756",2579326.0


In [149]:
A = A.drop('Unnamed: 0', 1)


ValueError: labels ['Unnamed: 0'] not contained in axis

In [151]:
A.columns = [x.lower() for x in A.columns]

In [152]:
out = A.to_json('../nbaApp/www/players.json', orient='records')